# Don't Use This

This was an attempt at merging the input data. I later realized that this can be done far more efficiently later on with the PyTorch dataset class.

In [ ]:
import os
import shutil
import cv2 as cv
import numpy as np
from tqdm.notebook import tqdm
import rasterio
import matplotlib.pyplot as plt

In [ ]:
def merge_tiffs(data_dir):
    os.makedirs(os.path.join(data_dir, "images_merge"), exist_ok=True)

    tiff_files_li=[]
    for ti in os.listdir(os.path.join(data_dir, "images/")):
        if '.tif' in ti:
            tiff_files_li.append(ti)
        else:
            shutil.copy(os.path.join(data_dir, "images/", ti), os.path.join(data_dir, "images_merge/", ti))
    
    for filename in tqdm(tiff_files_li):
        infile1 = os.path.join(data_dir,"images/",filename)
        infile2 = os.path.join(data_dir,"images2/",filename)
        outfile = os.path.join(data_dir,"images_merge/",filename)
        with rasterio.open(infile1) as dataset:
            # read only the first three bands (omitting NIR)
            data1 = dataset.read()[0:3,:,:]
            crs1 = dataset.crs
        with rasterio.open(infile2) as dataset:
            data2 = dataset.read()
            crs2 = dataset.crs
        
        # make sure image dimensions and crs are the same
        assert data1.shape[-2:] == data2.shape[-2:]
        assert crs1 == crs2

        # convert dtypes for dem
        # print(data2.dtype, np.min(data2), np.max(data2))
        data2 = data2 - np.min(data2)
        # print(data2.dtype, np.min(data2), np.max(data2))
        data2 = data2 / (np.max(data2) / 255)
        # print(data2.dtype, np.min(data2), np.max(data2))
        data2 = data2.astype(data1.dtype)
        # print(data2.dtype, np.min(data2), np.max(data2))

        # print(data1.shape, data2.shape)
        with rasterio.open(
            outfile,
            'w',
            driver='GTiff',
            count=data1.shape[0] + data2.shape[0],
            height=data1.shape[1],
            width=data1.shape[2],
            dtype=data1.dtype,
            crs=crs1,
            compress='lzw',
            PREDICTOR=2
            ) as new_dataset:
            # print("Output Shape:", new_dataset.shape)
            new_dataset.write(np.concatenate([data1, data2], axis=0))


In [ ]:
data_dir = "../data/Image_Chips_128_overlap_balanced_dem/"

In [ ]:
merge_tiffs(data_dir)

In [ ]:
testfile = None
for ti in os.listdir(os.path.join(data_dir, "images/")):
        if '.tif' in ti:
            testfile = ti
            break
testfile

In [ ]:
image1 = cv.imread(os.path.join(data_dir, "images/", testfile))
image1.shape

In [ ]:
image3 = cv.imread(os.path.join(data_dir, "images_merge/", testfile))
image3.shape

In [ ]:
fig, axs = plt.subplots(2, 4)
fig.suptitle("Original Image (top) and Merged Image (bottom)")
bands1 = ["B", "G", "R", "NIR"]
bands2 = ["B", "G", "R", "DEM"]
# axs[0, 3].imshow(image1[:,:,3])
# axs[1, 3].imshow(image3[:,:,3])
# image1_noalpha = image1[:,:,:3]
# image3_noalpha = image3[:,:,:3]
for i in range(3):
    axs[0, i].imshow(image1[:,:,i])
    axs[0, i].set_title(bands1[i])
    axs[0, i].set_axis_off()
    axs[1, i].imshow(image3[:,:,i])
    axs[1, i].set_title(bands2[i])
    axs[1, i].set_axis_off()